# Master Template

In [1]:
# imports
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import torch

# set numeric precison
pd.set_option("precision", 2)
torch.set_printoptions(precision=2, sci_mode=False)
%precision 2

# enable reload of changed files
%load_ext autoreload
%autoreload 2

# plot inline
%matplotlib inline

# Part 1

In [2]:
# talk about JSON and XML

In [3]:
# talk about HTTP methods: GET, POST, PUT, DELETE

In [4]:
# talk about serialization

In [5]:
import json

In [6]:
json.loads('{"name": "bob", "age" : 3}')

{'name': 'bob', 'age': 3}

In [7]:
# online JSON viewers
# jsonviewer.stack.hu
# jsonformatter.org/json-parser

# Part 2

In [9]:
import requests

In [10]:
r = requests.get('https://dog.ceo/api/breeds/image/random')

In [11]:
r.json() # content, headers, json, status_code, text, url

{'message': 'https://images.dog.ceo/breeds/terrier-dandie/n02096437_858.jpg',
 'status': 'success'}

In [12]:
from IPython.display import Image, display
from IPython.core.display import HTML 
import matplotlib.image as mpimg

In [13]:
for i in range(3):
    r = requests.get('https://dog.ceo/api/breeds/image/random')
    j = r.json()
    m = j['message']
    display(Image(url=m, width=300))

# Part 3

In [14]:
r = requests.get('https://api.covid19api.com/countries')

In [15]:
# json.loads(r.text)
countries = r.json()
len(countries)

248

In [16]:
names = np.empty((248), dtype='object')
names[:10]

array([None, None, None, None, None, None, None, None, None, None],
      dtype=object)

In [17]:
for i, c in enumerate(countries):
    names[i] = c['Country']

In [18]:
series = pd.Series(names, name='Countries')
series.head(3)

0      Senegal
1       Tuvalu
2    Gibraltar
Name: Countries, dtype: object

In [19]:
# top 10 countries in descending order
series.sort_values(ascending=False)[:10]

168                           Zimbabwe
95                              Zambia
219                              Yemen
223                     Western Sahara
180          Wallis and Futuna Islands
94                  Virgin Islands, US
241                           Viet Nam
122    Venezuela (Bolivarian Republic)
45                             Vanuatu
140                         Uzbekistan
Name: Countries, dtype: object

# Part 4

In [20]:
r = requests.get('https://api.covid19api.com/summary')
j = r.json()

In [21]:
df = pd.DataFrame(columns=['country', 'confirmed', 'deaths', 'recovered'])
for c in j['Countries']:
    name = c['Country']
    confirmed = c['TotalConfirmed']
    deaths = c['TotalDeaths']
    recovered = c['TotalRecovered']
    row = pd.Series([name, confirmed, deaths, recovered], index=['country', 'confirmed', 'deaths', 'recovered'])
    df = df.append(row, ignore_index=True)

In [22]:
df = df.astype({'country': 'object', 'confirmed' : 'int32', 'deaths' : 'int32', 'recovered' : 'int32'})

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   country    189 non-null    object
 1   confirmed  189 non-null    int32 
 2   deaths     189 non-null    int32 
 3   recovered  189 non-null    int32 
dtypes: int32(3), object(1)
memory usage: 3.8+ KB


In [24]:
df.head(5)

,country,confirmed,deaths,recovered
0,Afghanistan,40833,1514,34129
1,Albania,19157,477,10654
2,Algeria,56143,1914,39095
3,Andorra,4038,69,2729
4,Angola,9381,268,3508


In [25]:
# top 10 countries with worst covid death rate
df.sort_values(by=['deaths'], ascending=False)[:10]

,country,confirmed,deaths,recovered
180,United States of America,8635966,225229,3422878
23,Brazil,5380635,156903,4526393
76,India,7909959,119014,7137228
110,Mexico,891160,88924,752691
179,United Kingdom,876840,44986,2685
82,Italy,525782,37338,266203
159,Spain,1046132,34752,150376
59,France,1130143,34673,114534
132,Peru,888715,34149,803846
78,"Iran, Islamic Republic of",568896,32616,455054


# Part 5

In [26]:
r = requests.get('https://data.lacity.org/api/views/2nrs-mtv8/rows.json')
j = r.json()

In [27]:
len(j['data'])

155196

In [28]:
crimes = []
for x in j['data']:
    crime = x[17]
    crimes.append(crime)

In [29]:
crimes = pd.Series(crimes, name='crimes')
crimes[:10]

0                             BATTERY - SIMPLE ASSAULT
1                             BATTERY - SIMPLE ASSAULT
2      THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)
3             VANDALISM - MISDEAMEANOR ($399 OR UNDER)
4    VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...
5                                       RAPE, FORCIBLE
6             SHOPLIFTING - PETTY THEFT ($950 & UNDER)
7                            OTHER MISCELLANEOUS CRIME
8    THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...
9    VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...
Name: crimes, dtype: object

In [30]:
# top 10 crimes in LA
crimes.value_counts().sort_values(ascending=False).head(10)

VEHICLE - STOLEN                                           16175
BATTERY - SIMPLE ASSAULT                                   13325
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)    10324
BURGLARY FROM VEHICLE                                      10100
BURGLARY                                                   10014
ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT              9155
INTIMATE PARTNER - SIMPLE ASSAULT                           8752
THEFT PLAIN - PETTY ($950 & UNDER)                          8347
THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)             7796
VANDALISM - MISDEAMEANOR ($399 OR UNDER)                    5639
Name: crimes, dtype: int64

In [31]:
# how many were homicides
crimes[crimes.str.contains('hom', case=False)].value_counts()

CRIMINAL HOMICIDE    264
Name: crimes, dtype: int64